In [43]:
import numpy as np
# Load the training data
x_train = np.load('x_train.npy').reshape(-1, 28*28)

labels = np.load('y_train.npy')  # Assuming you have labels for the data
# Concatenate the data of the three classes
finalLabels = []
class_indices = {str(i): [] for i in range(3)}  # Assuming you have only 3 classes
for i, label in enumerate(labels):
    if (label < 3):  # Considering only the first 3 classes
        class_indices[str(label)].append(i)

data_class_0 = x_train[class_indices['0']]
data_class_1 = x_train[class_indices['1']]
data_class_2 = x_train[class_indices['2']]

label_class_0 = labels[class_indices['0']]
label_class_1 = labels[class_indices['1']]
label_class_2 = labels[class_indices['2']]

labels = np.concatenate((label_class_0, label_class_1, label_class_2), axis=0)


# Concatenate data from all three classes
X = np.concatenate((data_class_0, data_class_1, data_class_2), axis=0)
print(X.shape)


(18623, 784)


In [54]:
def pca(data, n_components=10):
    # Step 1: Calculate mean of the data
    data_mean = np.mean(data, axis=0)
    
    # Step 2: Center the data
    data_centered = data - data_mean
    
    # Step 3: Compute covariance matrix
    covariance_matrix = np.cov(data_centered, rowvar=False)
    
    # Step 4: Compute eigenvectors and eigenvalues
    eigenvalues, eigenvectors = np.linalg.eigh(covariance_matrix)
    
    # Step 5: Sort eigenvectors and eigenvalues in descending order
    sorted_indices = np.argsort(eigenvalues)[::-1]
    sorted_eigenvalues = eigenvalues[sorted_indices]
    sorted_eigenvectors = eigenvectors[:, sorted_indices]
    
    # Step 6: Choose top n_components eigenvectors
    U = sorted_eigenvectors[:, :n_components]

    data_reduced = np.dot(data_centered,U)
    
    return data_reduced
# pca_matrix1, x_reconstructed = pca(X, n_components=10)
# pca_matrix = np.dot(pca_matrix1, x_reconstructed).T
pca_matrix = pca(X, n_components=10)
print(pca_matrix.shape)


(18623, 10)


In [45]:
class Node():
    def __init__(self, feature_index=None, threshold=None, left=None, right=None, info_gain=None, value=None):
        ''' constructor ''' 
        
        # for decision node
        self.feature_index = feature_index
        self.threshold = threshold
        self.left = left
        self.right = right
        self.info_gain = info_gain

In [46]:
class DecisionTreeClassifier():
    def __init__(self, min_samples_split=2, max_depth=2):
        ''' constructor '''
        
        # initialize the root of the tree 
        self.root = None
        
        # stopping conditions
        self.min_samples_split = min_samples_split
        self.max_depth = max_depth
        
    def build_tree(self, dataset, curr_depth=0):
        ''' recursive function to build the tree ''' 
        
        X, Y = dataset[:,:-1], dataset[:,-1]
        num_samples, num_features = np.shape(X)
        
        # split until stopping conditions are met
        if num_samples>=self.min_samples_split and curr_depth<=self.max_depth:
            # find the best split
            best_split = self.get_best_split(dataset, num_samples, num_features)
            # check if information gain is positive
            if best_split["info_gain"]>0:
                # recur left
                left_subtree = self.build_tree(best_split["dataset_left"], curr_depth+1)
                # recur right
                right_subtree = self.build_tree(best_split["dataset_right"], curr_depth+1)
                # return decision node
                return Node(best_split["feature_index"], best_split["threshold"], 
                            left_subtree, right_subtree, best_split["info_gain"])
        
        # compute leaf node
        leaf_value = self.calculate_leaf_value(Y)
        # return leaf node
        return Node(value=leaf_value)
    
    def get_best_split(self, dataset, num_samples, num_features):
        ''' function to find the best split '''
        
        # dictionary to store the best split
        best_split = {}
        max_info_gain = -float("inf")
        
        # loop over all the features
        for feature_index in range(num_features):
            feature_values = dataset[:, feature_index]
            possible_thresholds = np.unique(feature_values)
            # loop over all the feature values present in the data
            for threshold in possible_thresholds:
                # get current split
                dataset_left, dataset_right = self.split(dataset, feature_index, threshold)
                # check if childs are not null
                if len(dataset_left)>0 and len(dataset_right)>0:
                    y, left_y, right_y = dataset[:, -1], dataset_left[:, -1], dataset_right[:, -1]
                    # compute information gain
                    curr_info_gain = self.information_gain(y, left_y, right_y, "gini")
                    # update the best split if needed
                    if curr_info_gain>max_info_gain:
                        best_split["feature_index"] = feature_index
                        best_split["threshold"] = threshold
                        best_split["dataset_left"] = dataset_left
                        best_split["dataset_right"] = dataset_right
                        best_split["info_gain"] = curr_info_gain
                        max_info_gain = curr_info_gain
                        
        # return best split
        return best_split
    
    def split(self, dataset, feature_index, threshold):
        ''' function to split the data '''
        
        dataset_left = np.array([row for row in dataset if row[feature_index]<=threshold])
        dataset_right = np.array([row for row in dataset if row[feature_index]>threshold])
        return dataset_left, dataset_right
    
    def information_gain(self, parent, l_child, r_child, mode="entropy"):
        ''' function to compute information gain '''
        
        weight_l = len(l_child) / len(parent)
        weight_r = len(r_child) / len(parent)
        if mode=="gini":
            gain = self.gini_index(parent) - (weight_l*self.gini_index(l_child) + weight_r*self.gini_index(r_child))
        else:
            gain = self.entropy(parent) - (weight_l*self.entropy(l_child) + weight_r*self.entropy(r_child))
        return gain
    
    def entropy(self, y):
        ''' function to compute entropy '''
        
        class_labels = np.unique(y)
        entropy = 0
        for cls in class_labels:
            p_cls = len(y[y == cls]) / len(y)
            entropy += -p_cls * np.log2(p_cls)
        return entropy
    
    def gini_index(self, y):
        ''' function to compute gini index '''
        
        class_labels = np.unique(y)
        gini = 0
        for cls in class_labels:
            p_cls = len(y[y == cls]) / len(y)
            gini += p_cls**2
        return 1 - gini
        
    def calculate_leaf_value(self, Y):
        ''' function to compute leaf node '''
        
        Y = list(Y)
        return max(Y, key=Y.count)
    
    def print_tree(self, tree=None, indent=" "):
        ''' function to print the tree '''
        
        if not tree:
            tree = self.root

        if tree.value is not None:
            print(tree.value)

        else:
            print("X_"+str(tree.feature_index), "<=", tree.threshold, "?", tree.info_gain)
            print("%sleft:" % (indent), end="")
            self.print_tree(tree.left, indent + indent)
            print("%sright:" % (indent), end="")
            self.print_tree(tree.right, indent + indent)
    
    def fit(self, X, Y):
        ''' function to train the tree '''
        
        dataset = np.concatenate((X, Y), axis=1)
        self.root = self.build_tree(dataset)
    
    def predict(self, X):
        ''' function to predict new dataset '''
        predictions = [self.make_prediction(x, self.root) for x in X]
        return predictions

    def make_prediction(self, x, tree):
        ''' function to predict a single data point '''
        if tree.value is not None:
            return tree.value  # If it's a leaf node, return the value
        feature_val = x[tree.feature_index]
        if feature_val <= tree.threshold:
            return self.make_prediction(x, tree.left)
        else:
            return self.make_prediction(x, tree.right)


In [47]:
print(pca_matrix.shape)
y_train = labels.reshape(-1, 1)
print(y_train.shape)

(18623, 10)
(18623, 1)


In [52]:

classifier = DecisionTreeClassifier(min_samples_split=3, max_depth=2)
classifier.fit(pca_matrix[:1000],y_train[:1000])
classifier.print_tree()

AttributeError: 'Node' object has no attribute 'value'

In [49]:
# Load the test data
x_test = np.load('x_test.npy').reshape(-1, 28*28)
y_test = np.load('y_test.npy')

# Filter out samples from classes other than 0, 1, and 2
class_indices_test = {str(i): [] for i in range(3)}
for i, label in enumerate(y_test):
    if label < 3:  # Considering only the first 3 classes
        class_indices_test[str(label)].append(i)

# Extract data corresponding to classes 0, 1, and 2
data_class_0_test = x_test[class_indices_test['0']]
data_class_1_test = x_test[class_indices_test['1']]
data_class_2_test = x_test[class_indices_test['2']]

# Concatenate data from all three classes vertically
X_test = np.concatenate((data_class_0_test, data_class_1_test, data_class_2_test), axis=0)
y_test_filtered = y_test[np.concatenate((class_indices_test['0'], class_indices_test['1'], class_indices_test['2']))]

# Calculate accuracy


In [50]:
X_test_pca = pca(X_test, n_components=10)

In [51]:
Y_pred = classifier.predict(np.real(X_test_pca)) 
from sklearn.metrics import accuracy_score
accuracy_score(y_test_filtered, np.real(Y_pred))

AttributeError: 'Node' object has no attribute 'value'